In [1]:
from stats import *
import os
import numpy as np

In [2]:
def get_class_label(tag:int, label:np.array(float)):
    gt = label.copy()
    for i in range(len(gt)):
        if gt[i]==tag : 
            gt[i]=1
        else:
            gt[i]=0
    return gt

In [3]:
dir_base = '/home/nguyen-k/Bureau/segCassiopet2/Comparatif'
dir_1 = os.path.join(dir_base,'Archives_reunion/Multitask_PET_L0')
dir_2 = os.path.join(dir_base,'Archives_4diffuse_noSeg/Multitask_PET_L0')
dir_3 = os.path.join(dir_base,'Archives_4diffuse/Multitask_PET_L0')
dir_4 = os.path.join(dir_base,'Archives_Encodeur/Encodeur_PET_L0')

best_fold_1 = 'Fold4'
best_fold_2 = 'Fold4'
best_fold_3 = 'Fold5'
best_fold_4 = 'Fold5'

In [4]:
# get label
ground_truth_ini = np.load('/home/nguyen-k/Bureau/segCassiopet2/List_Patient_0/Test/test_label_classe.npy') 

# get proba prediction of best fold 
prob_1_ini = np.load(os.path.join(dir_1,best_fold_1,'test_prob.npy'))
prob_2_ini = np.load(os.path.join(dir_2,best_fold_2,'test_prob.npy'))
prob_3_ini = np.load(os.path.join(dir_3,best_fold_3,'test_prob.npy'))
prob_4_ini = np.load(os.path.join(dir_4,best_fold_4,'test_prob.npy'))

In [5]:
# function to compute p-value using Delong method
def get_p_value(tag,ground_truth_ini,prob_1_ini,prob_2_ini,prob_3_ini,prob_4_ini):
    ground_truth = get_class_label(tag,ground_truth_ini)
    prob_1 = prob_1_ini[:,tag]
    prob_2 = prob_2_ini[:,tag]
    prob_3 = prob_3_ini[:,tag]
    prob_4 = prob_4_ini[:,tag]
    pv_log10 = delong_roc_test(ground_truth, prob_1, prob_2)
    pv_12 = 10**pv_log10[0][0]
    pv_log10 = delong_roc_test(ground_truth, prob_3, prob_2)
    pv_23 = 10**pv_log10[0][0]
    pv_log10 = delong_roc_test(ground_truth, prob_1, prob_3)
    pv_13 = 10**pv_log10[0][0]
    pv_log10 = delong_roc_test(ground_truth, prob_1, prob_4)
    pv_14 = 10**pv_log10[0][0]
    pv_log10 = delong_roc_test(ground_truth, prob_2, prob_4)
    pv_24 = 10**pv_log10[0][0]
    pv_log10 = delong_roc_test(ground_truth, prob_3, prob_4)
    pv_34 = 10**pv_log10[0][0]
    return pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 

In [6]:
# get p-value for class 2 (treatment response) : best fold method
tag=2
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,prob_1_ini,prob_2_ini,prob_3_ini,prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)

# get p-value for class 1 (treatment response) : best fold method
tag=1
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,prob_1_ini,prob_2_ini,prob_3_ini,prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)

# get p-value for class 0 (treatment response) : best fold method
tag=0
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,prob_1_ini,prob_2_ini,prob_3_ini,prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)



class =  2 

aucs  [0.61594203 0.92028986]
aucs  [0.88405797 0.92028986]
aucs  [0.61594203 0.88405797]
aucs  [0.61594203 0.84057971]
aucs  [0.92028986 0.84057971]
aucs  [0.88405797 0.84057971]
0.23246367528973444 0.4156632948224017 0.22172606779904402 0.4822897583220659 0.4966229430678623 0.7559543091882874

class =  1 

aucs  [0.60526316 0.6291866 ]
aucs  [0.53349282 0.6291866 ]
aucs  [0.60526316 0.53349282]
aucs  [0.60526316 0.49043062]
aucs  [0.6291866  0.49043062]
aucs  [0.53349282 0.49043062]
0.8480784547594431 0.3384915300384115 0.5378570008544382 0.314646572827773 0.3115152134649586 0.7244051812074813

class =  0 

aucs  [0.64897959 0.6877551 ]
aucs  [0.59795918 0.6877551 ]
aucs  [0.64897959 0.59795918]
aucs  [0.64897959 0.57959184]
aucs  [0.6877551  0.57959184]
aucs  [0.59795918 0.57959184]
0.7117641249049868 0.33477354950264265 0.6079018291475219 0.4989432336207029 0.31755637840862727 0.8571024252977725


In [7]:
# get average probability prediction over 5 folds
def get_avg_prob(dir):
    fold = 1
    dir_p = dir+'/Fold'+str(fold)
    sum = np.load(os.path.join(dir_p, 'test_prob.npy'))


    for fold in range(2, 6) :
        dir_p = dir+'/Fold'+str(fold)
        test_prob = np.load(os.path.join(dir_p, 'test_prob.npy'))
        sum += test_prob

    test_prob = sum / 5
    return test_prob

In [8]:
# get average prob for diferent methods
avg_prob_1_ini = get_avg_prob(dir_1)
avg_prob_2_ini = get_avg_prob(dir_2)
avg_prob_3_ini = get_avg_prob(dir_3)
avg_prob_4_ini = get_avg_prob(dir_4)

# get p-value for class 2 (treatment response)
tag=2
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,avg_prob_1_ini,avg_prob_2_ini,avg_prob_3_ini,avg_prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)

# get p-value for class 1 (treatment response)
tag=1
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,avg_prob_1_ini,avg_prob_2_ini,avg_prob_3_ini,avg_prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)

# get p-value for class 0 (treatment response)
tag=0
print('\nclass = ',tag,'\n')
pv_12, pv_23, pv_13, pv_14, pv_24, pv_34 = get_p_value(tag,ground_truth_ini,avg_prob_1_ini,avg_prob_2_ini,avg_prob_3_ini,avg_prob_4_ini)
print(pv_12,pv_23,pv_13,pv_14,pv_24,pv_34)


class =  2 

aucs  [0.79710145 0.81884058]
aucs  [0.82608696 0.81884058]
aucs  [0.79710145 0.82608696]
aucs  [0.79710145 0.73913043]
aucs  [0.81884058 0.73913043]
aucs  [0.82608696 0.73913043]
0.6284973737353075 0.9362858465813705 0.7460325948660228 0.5980732069081387 0.39836490321621726 0.6272191181223329

class =  1 

aucs  [0.59569378 0.58133971]
aucs  [0.50239234 0.58133971]
aucs  [0.59569378 0.50239234]
aucs  [0.59569378 0.38995215]
aucs  [0.58133971 0.38995215]
aucs  [0.50239234 0.38995215]
0.8562498676415088 0.20040701948230688 0.11483956912221796 0.06819354849499998 0.06329472796003852 0.23186514517559403

class =  0 

aucs  [0.64285714 0.67959184]
aucs  [0.62040816 0.67959184]
aucs  [0.64285714 0.62040816]
aucs  [0.64285714 0.5244898 ]
aucs  [0.67959184 0.5244898 ]
aucs  [0.62040816 0.5244898 ]
0.6022834668606145 0.24733845924458095 0.7265761505364254 0.22007881866716691 0.038564439197176566 0.1614384545252491
